In [1]:
import xarray as xr
import glob
import numpy as np
import matplotlib.pyplot as plt
from utils import *

In [15]:
def fix_time(ds):
    yr0=str(ds['time.year'][0].values)
    nt=len(ds.time)
    ds['time'] = xr.cftime_range(yr0,periods=nt,freq='MS',calendar='noleap') #fix time bug
    return ds

In [2]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
def get_cluster(workers=30,project = 'P93300041'):

    cluster = PBSCluster(
        cores=1, # The number of cores you want
        memory='10GB', # Amount of memory
        processes=1, # How many processes
        queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
        local_directory='$TMPDIR', # Use your local directory
        resource_spec='select=1:ncpus=1:mem=10GB', # Specify resources
        account=project, # Input your project ID here
        walltime='01:30:00', # Amount of wall time
    )
    
    # Scale up
    cluster.scale(workers)
    
    # Setup your client
    client = Client(cluster)
    return client

In [3]:
client=get_cluster()

In [4]:
client.cluster

PBSCluster(3d251823, 'tcp://128.117.208.103:37085', workers=30, threads=30, memory=279.30 GiB)

### gridded data

In [7]:
d='/glade/derecho/scratch/slevis/archive/ctsm53017_f19_BNF_hist/lnd/hist/'
files=sorted(glob.glob(d+'*h0*'))

In [12]:
dvs=['RAIN','SNOW','Q2M','FSDS','TOTECOSYSC','TOTECOSYSN','TOTSOILICE','TOTSOILLIQ',
     'FSNO','HR','NBP','QRUNOFF','FCEV','FGEV']
def pp(ds):
    return ds[dvs]

In [ ]:
ds=xr.open_mfdataset(files,combine='by_coords',parallel=True,preprocess=pp)
ds=fix_time(ds)
ix=ds['time.year']>=1920
ds=ds.isel(time=ix)

In [19]:
g1=xr.DataArray(np.repeat(np.arange(4),26),dims=['year'],name='epoch')
g2=xr.DataArray(np.repeat(np.arange(4),26*12),dims=['time'],name='epoch')

In [ ]:
dsout=xr.Dataset()
for v in dvs:
    print(v)
    x=amean(ds[v]).groupby(g1).mean().compute()
    dsout[v+'_mean']=x
    x=ds[v].groupby(g2).std().compute()
    dsout[v+'_std']=x

In [41]:
dsout['yr1']=ds['time.year'].groupby(g2).min()
dsout['yr2']=ds['time.year'].groupby(g2).max()
dsout.attrs={'data_dir':d,
             'script':'/glade/u/home/djk2120/ctsm6_ppe/sparsegrid/fullgrid_features.ipynb'}
dsout.to_netcdf('fullgrid/ctsm53017_f19_BNF_gc_metrics.nc')

### pft-level data

In [2]:
d='/glade/derecho/scratch/slevis/archive/ctsm53017_f19_BNF_hist/lnd/hist/'
files=sorted(glob.glob(d+'*h1*'))

In [5]:
ds=xr.open_mfdataset(files,combine='by_coords',parallel=True)
ds=fix_time(ds)

In [84]:
g1=xr.DataArray(np.repeat(np.arange(4),26),dims=['year'],name='epoch')
g2=xr.DataArray(np.repeat(np.arange(4),26*12),dims=['time'],name='epoch')

In [85]:
dvs=['GPP','AR','TSA','TOTVEGC','TLAI','FCTR']

In [86]:
dpm=ds['time.daysinmonth']

In [87]:
def amean(da):
    #annual mean of monthly data
    m  = da['time.daysinmonth']
    cf = 1/365
    xa = cf*(m*da).groupby('time.year').sum().compute()
    return xa

In [88]:
dsout=xr.Dataset()
for v in dvs:
    print(v)
    x=amean(ds[v]).groupby(g1).mean().compute()
    dsout[v+'_mean']=x
    x=ds[v].groupby(g2).std().compute()
    dsout[v+'_std']=x

GPP
AR
TSA
TOTVEGC
TLAI
FCTR


In [102]:
dsout['yr1']=ds['time.year'].groupby(g2).min()
dsout['yr2']=ds['time.year'].groupby(g2).max()
dsout.attrs={'data_dir':d,
             'script':'/glade/u/home/djk2120/ctsm6_ppe/sparsegrid/fullgrid_features.ipynb'}
dsout.to_netcdf('fullgrid/ctsm53017_f19_BNF_pft_metrics.nc')